In [37]:
import pandas as pd
from datetime import date, timedelta, datetime



In [39]:
all_raw_data = pd.read_csv("/Users/allenchen/projects/baseball-analytics/data/aggregated/20230504_agg_raw_data_small.csv")

In [40]:
all_raw_data.shape

(9034, 23)

In [43]:
all_raw_data.sort_values("game_date").tail()

,game_date,batter,pitcher,events,description,game_type,stand,p_throws,home_team,away_team,...,on_1b,outs_when_up,inning,inning_topbot,launch_speed,at_bat_number,pitch_number,game_pk,bat_score,fld_score
22,2023-05-03,575929,660271,d,hit_into_play,R,R,R,STL,LAA,...,0,0,4,Bot,97.4,32,1,718320,1,3
21,2023-05-03,676475,660271,so,swinging_strike,R,L,R,STL,LAA,...,0,0,4,Bot,NaN,33,4,718320,2,3
20,2023-05-03,666185,660271,hr,hit_into_play,R,L,R,STL,LAA,...,0,1,4,Bot,103.5,34,2,718320,2,3
18,2023-05-03,669242,660271,fo,hit_into_play,R,L,R,STL,LAA,...,0,2,4,Bot,74.4,36,3,718320,4,3
0,2023-05-03,669242,608032,fo,hit_into_play,R,L,R,STL,LAA,...,0,2,9,Bot,72.1,77,5,718320,4,6


In [42]:
all_raw_data[all_raw_data.on_3b.isnull()]

,game_date,batter,pitcher,events,description,game_type,stand,p_throws,home_team,away_team,...,on_1b,outs_when_up,inning,inning_topbot,launch_speed,at_bat_number,pitch_number,game_pk,bat_score,fld_score


In [30]:
date(2008, 2, 27) + timedelta(365)

datetime.date(2009, 2, 26)

In [38]:
datetime.combine(date.today(), datetime.min.time())

datetime.datetime(2023, 5, 12, 0, 0)

In [5]:
#all_raw_data = all_raw_data.astype({"batter": str, "pitcher": str, "on_3b": str, "on_2b": str, "on_1b": str, "game_pk": str})

In [7]:
all_raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3158769 entries, 0 to 3158768
Data columns (total 23 columns):
 #   Column         Dtype  
---  ------         -----  
 0   game_date      object 
 1   batter         object 
 2   pitcher        object 
 3   events         object 
 4   description    object 
 5   game_type      object 
 6   stand          object 
 7   p_throws       object 
 8   home_team      object 
 9   away_team      object 
 10  game_year      int64  
 11  on_3b          object 
 12  on_2b          object 
 13  on_1b          object 
 14  outs_when_up   int64  
 15  inning         int64  
 16  inning_topbot  object 
 17  launch_speed   float64
 18  at_bat_number  int64  
 19  pitch_number   int64  
 20  game_pk        object 
 21  bat_score      int64  
 22  fld_score      int64  
dtypes: float64(1), int64(7), object(15)
memory usage: 554.3+ MB


In [6]:
from pyspark.sql.functions import pandas_udf, PandasUDFType, col
from pyspark.sql.types import *
from pyspark.sql import SparkSession

In [8]:
spark = (
    SparkSession
    .builder
    .master("local[12]")
    .appName("baseball-analytics")
    .getOrCreate()
)

23/05/12 16:47:39 WARN Utils: Your hostname, Allens-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.66 instead (on interface en0)
23/05/12 16:47:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/12 16:47:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/12 16:47:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [9]:
raw_data_schema = StructType(
    [
        StructField('game_date', StringType(), True), 
        StructField('batter', StringType(), True), 
        StructField('pitcher', StringType(), True), 
        StructField('events', StringType(), True), 
        StructField('description', StringType(), True), 
        StructField('game_type', StringType(), True), 
        StructField('stand', StringType(), True), 
        StructField('p_throws', StringType(), True), 
        StructField('home_team', StringType(), True), 
        StructField('away_team', StringType(), True), 
        StructField('game_year', ShortType(), True), 
        StructField('on_3b', StringType(), True), 
        StructField('on_2b', StringType(), True), 
        StructField('on_1b', StringType(), True), 
        StructField('outs_when_up', ByteType(), True), 
        StructField('inning', ByteType(), True), 
        StructField('inning_topbot', StringType(), True), 
        StructField('launch_speed', DoubleType(), True), 
        StructField('at_bat_number', ShortType(), True), 
        StructField('pitch_number', ShortType(), True), 
        StructField('game_pk', StringType(), True), 
        StructField('bat_score', ByteType(), True), 
        StructField('fld_score', ByteType(), True), 
    ]
) 

data = (
    spark
    .read
    .format("csv")
    .option("header", "true")
    .schema(raw_data_schema)
    .load("/Users/allenchen/projects/baseball-analytics/data/aggregated/20230504_agg_raw_data_small.csv")
)

In [10]:
data.show()

+----------+------+-------+------+---------------+---------+-----+--------+---------+---------+---------+------+-----+------+------------+------+-------------+------------+-------------+------------+-------+---------+---------+
| game_date|batter|pitcher|events|    description|game_type|stand|p_throws|home_team|away_team|game_year| on_3b|on_2b| on_1b|outs_when_up|inning|inning_topbot|launch_speed|at_bat_number|pitch_number|game_pk|bat_score|fld_score|
+----------+------+-------+------+---------------+---------+-----+--------+---------+---------+---------+------+-----+------+------------+------+-------------+------------+-------------+------------+-------+---------+---------+
|2023-05-03|669242| 608032|    fo|  hit_into_play|        R|    L|       R|      STL|      LAA|     2023|     0|    0|     0|           2|     9|          Bot|        72.1|           77|           5| 718320|        4|        6|
|2023-05-03|680977| 608032|    so|swinging_strike|        R|    L|       R|      STL|   

In [13]:
dict(R=1, L=0)

{'R': 1, 'L': 0}